In [ ]:
from delta.tables import *
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, DataType, BooleanType, NumericType, TimestampType, IntegerType, DecimalType, StringType, LongType, DateType, FloatType
from botocore.client import Config
from pyspark.sql.functions import udf

import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import groupby
from matplotlib.ticker import StrMethodFormatter, PercentFormatter
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns

### Define and Modify API Key Variables

In [ ]:
AMBERDATA_API_KEY = "INSERT_YOUR_AMBERDATA_API_KEY_HERE"
GVOL_API_KEY = "INSERT_YOUR_AD_DERIVATIVES_API_KEY_HERE"

### Import OHLCV Data From Amberdata API

In [ ]:
# Define start and end dates
start_date = datetime.strptime("2022-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-08-31", "%Y-%m-%d")

# Initialize empty DataFrame
final_df = pd.DataFrame()

# Create date ranges
date_ranges = []
while start_date < end_date:
    new_end_date = start_date + timedelta(days=30)
    date_ranges.append((start_date, new_end_date))
    start_date = new_end_date

# Loop through date ranges to get data
for start, end in date_ranges:
    url = f"https://web3api.io/api/v2/market/spot/ohlcv/exchange/bitfinex/historical?pair=usdc_usdt&timeInterval=days&startDate={start.strftime('%Y-%m-%dT%H:%M:%S')}&endDate={end.strftime('%Y-%m-%dT%H:%M:%S')}"
    
    headers = {
        "accept": "application/json",
        "x-api-key": AMBERDATA_API_KEY
    }
    
    # Make the API request
    response = requests.get(url, headers=headers)
    
    # Parse the JSON data into a Python dictionary
    data = response.json()
    
    # Get the relevant data for 'usdc_usdt' pair
    usdc_usdt_data = data['payload']['data']['usdc_usdt']
    
    # Get the column names from metadata
    columns = data['payload']['metadata']['columns']
    
    # Create a Pandas DataFrame with the data and column names
    df = pd.DataFrame(usdc_usdt_data, columns=columns)
    
    # Convert 'timestamp' to datetime format (assuming the timestamp is in milliseconds)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # Set 'timestamp' as the DataFrame's index
    df.set_index('timestamp', inplace=True)
    
    # Append to final DataFrame
    final_df = pd.concat([final_df, df])

# Display the final DataFrame
final_df

### Plot Close Prices

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Define Amberdata color theme
AD_COLORS_1 = ['#1c3664', '#89bed8', '#f16623']

# Create a single subplot (1 row, 1 column)
fig = make_subplots(rows=1, cols=1)

# Line plot of close prices
fig.add_trace(go.Scatter(
    x=final_df.index,
    y=final_df['close'],
    mode='lines',
    name='Close',
    line=dict(color=AD_COLORS_1[0])  # Set line color
))

# Update layout to match the theme
fig.update_layout(
    title='USDC/USDT Close Prices Over Time (Bitfinex)',
    yaxis_title="Close",
    template='plotly_white',
    height=600,
    showlegend=True
)

# Remove x-axis label
fig.update_layout(xaxis_title=None)

# Show the plot
fig.show()

### Plot Close Prices vs. Volume

In [ ]:
# Create a two-row subplot (2 rows, 1 column) with custom height ratios
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Close Prices", "Volume"), vertical_spacing=0.1, row_heights=[0.7, 0.3])

# Line plot of close prices on the first subplot
fig.add_trace(
    go.Scatter(
        x=final_df.index,
        y=final_df['close'],
        mode='lines',
        name='Close',
        line=dict(color=AD_COLORS_1[0])  # Set line color
    ),
    row=1, col=1
)

# Bar plot of volume on the second subplot
fig.add_trace(
    go.Bar(
        x=final_df.index,
        y=final_df['volume'],
        name='Volume',
        marker=dict(color=AD_COLORS_1[1])  # Set bar color
    ),
    row=2, col=1
)

# Update layout to match the theme
fig.update_layout(
    title='USDC/USDT Close Prices and Volume Over Time (Bitfinex)',
    template='plotly_white',
    height=800,
    showlegend=True
)

# Update y-axis labels
fig.update_yaxes(title_text="Close Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)

fig.show()

### Calculate Bid-Ask Spread

In [ ]:
# Window specification for timestamp
timestampWindow = Window.partitionBy("exchangeTimestamp")

# Window specification for bid prices
bid_window = Window.partitionBy('exchangeTimestamp')\
        .orderBy(f.desc('bid_price'))\
        .rowsBetween(Window.unboundedPreceding, 0)

# Window specification for ask prices
ask_window = Window.partitionBy('exchangeTimestamp')\
        .orderBy(f.asc('ask_price'))\
        .rowsBetween(Window.unboundedPreceding, 0)

# Convert exchangeTimestamp to datetime format
bid_book = bid_book.withColumn("exchangeTimestamp", f.from_unixtime(col("exchangeTimestamp") / 1000).cast("timestamp"))
ask_book = ask_book.withColumn("exchangeTimestamp", f.from_unixtime(col("exchangeTimestamp") / 1000).cast("timestamp"))

# Get maximum bid price and minimum ask price for each exchangeTimestamp
bid_book = bid_book\
        .withColumn("max_bid", f.max("bid_price").over(timestampWindow))

ask_book = ask_book\
        .withColumn("min_ask", f.min("ask_price").over(timestampWindow))

# Join bid_book and ask_book on exchangeTimestamp
df = bid_book.join(ask_book, on='exchangeTimestamp', how='inner')

# Calculate bid-ask spread for each exchangeTimestamp
df = df.withColumn("bid_ask_spread", df.min_ask - df.max_bid)

# Group by exchangeTimestamp to get a single bid-ask spread per timestamp
df = df.groupby("exchangeTimestamp")\
            .agg(f.first("bid_ask_spread", ignorenulls=True).alias("bid_ask_spread"))

# Convert the Spark DataFrame to a Pandas DataFrame
df = df.toPandas()

# Set 'timestamp' as the DataFrame's index
df.set_index('exchangeTimestamp', inplace=True)
df.sort_index(inplace=True)
df

### Plot Daily Aggregated Bid-Ask Spread

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

# Resample to daily frequency, taking the mean
df_daily = df['bid_ask_spread'].resample('D').mean()

# Create a two-row subplot (2 rows, 1 column)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Close Prices", "Daily Mean Bid-Ask Spread"), vertical_spacing=0.1)

# Line plot of close prices on the first subplot
fig.add_trace(
    go.Scatter(
        x=final_df.index,
        y=final_df['close'],
        mode='lines',
        name='Close',
        line=dict(color=AD_COLORS_1[0])
    ),
    row=1, col=1
)

# Bar plot of daily mean bid-ask spread on the second subplot
fig.add_trace(
    go.Bar(
        x=df_daily.index,
        y=df_daily,
        name='Daily Mean Bid-Ask Spread',
        marker=dict(color=AD_COLORS_1[1])
    ),
    row=2, col=1
)

# Update layout to match the theme
fig.update_layout(
    title='USDC/USDT Close Prices and Daily Mean Bid-Ask Spread Over Time',
    template='plotly_white',
    height=800,
    showlegend=True
)

# Update y-axis labels
fig.update_yaxes(title_text="Close Price", row=1, col=1)
fig.update_yaxes(title_text="Daily Mean Bid-Ask Spread", row=2, col=1)

# Show the plot
fig.show()

### Import Orderbook Data

In [ ]:
df = (
    DeltaTable
    .forPath(spark, 's3://amberdata-marketdata-deltalake/spot/order-book-snapshots/').toDF()
    .where('year = "2023" \
            AND month = "03"\
            AND day IN ("10", "11", "12")\
            AND pair = "usdc_usdt"')
    .select(f.explode('orderBookSides')).select("col.exchangeTimestamp", "col.isBid", f.explode('col.data'))
).cache()

prices = (
    DeltaTable
    .forPath(spark, 's3://amberdata-marketdata-deltalake/spot/order-book-snapshots/').toDF()
    .where('year = "2023" \
            AND month = "03"\
            AND day IN ("10", "11", "12")\
            AND exchange = "bitfinex"\
            AND pair = "usdc_usdt"')
    .select(f.explode('orderBookSides')).select("col.exchangeTimestamp", "col.isBid", f.explode('col.data'))
).cache()

# Enable arrow for faster Pandas conversion
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Convert to Pandas DataFrames
df = df.toPandas()
prices = prices.toPandas()

### Orderbook Heatmap Aggregations

In [ ]:
df['exchangeTimestamp'] = pd.to_datetime(df['exchangeTimestamp'], unit='ms')
df['exchangeTimestamp'] = df['exchangeTimestamp'].dt.round('min')

df = df.assign(price=lambda x: [val[0] for val in x['col'].values],
               quantity=lambda x: [val[1] for val in x['col'].values])

df = df.drop(['col'], axis=1)
df.rename(columns={'exchangeTimestamp':'timestamp'}, inplace=True)

bid_book = df[df['isBid']==True]
ask_book = df[df['isBid']==False]

ask_book = ask_book.sort_values(by='timestamp')
bid_book = bid_book.sort_values(by='timestamp')

prices['exchangeTimestamp'] = pd.to_datetime(prices['exchangeTimestamp'], unit='ms')
prices['exchangeTimestamp'] = prices['exchangeTimestamp'].dt.round('min')

prices = prices.assign(price=lambda x: [val[0] for val in x['col'].values],
               quantity=lambda x: [val[1] for val in x['col'].values])

prices = prices.drop(['col'], axis=1)
prices.rename(columns={'exchangeTimestamp':'timestamp'}, inplace=True)

bid_book_prices = prices[prices['isBid']==True]
ask_book_prices = prices[prices['isBid']==False]

ask_book_prices = ask_book_prices.sort_values(by='timestamp')
bid_book_prices = bid_book_prices.sort_values(by='timestamp')

lowest_ask_group = ask_book_prices.groupby('timestamp').agg({'price': 'min'})
lowest_ask = lowest_ask_group.rename(columns={'price': 'lowest_ask'})

highest_bid_group = bid_book_prices.groupby('timestamp').agg({'price': 'max'})
highest_bid = highest_bid_group.rename(columns={'price': 'highest_bid'})

prices = pd.merge(left=lowest_ask, right=highest_bid, on='timestamp')
prices['mid_price'] = (prices['lowest_ask'] + prices['highest_bid']) / 2

prices = prices.reset_index()

In [ ]:
def rounded_order_book_heatmap(ask_book,
                       bid_book,
                       min_price = 1500,
                       max_price = 1700,
                       rounding_factor=1,
                       startDate = '2023-02-24 0:00:00',
                       endDate = '2023-02-24 1:00:00',
                       title='Order Book Heatmap'
                      ):
    
    ask_book = ask_book.copy()
    bid_book = bid_book.copy()
    
    ask_book['price'] = np.floor(ask_book['price'] / rounding_factor) * rounding_factor

    # Group the data by timestamp and price
    ask_book = ask_book.groupby(['timestamp', 'price'])

    # Aggregate the sum of the quantity
    ask_book = ask_book.agg({'quantity': 'sum'})

    # Reset the index
    ask_book = ask_book.reset_index()

    bid_book['price'] = np.ceil(bid_book['price'] / rounding_factor) * rounding_factor

    # Group the data by timestamp and price
    bid_book = bid_book.groupby(['timestamp', 'price'])

    # Aggregate the sum of the quantity
    bid_book = bid_book.agg({'quantity': 'sum'})

    # Reset the index
    bid_book = bid_book.reset_index()

    orderbook = ask_book.append(bid_book)

    # Group the data by timestamp and price
    orderbook = orderbook.groupby(['timestamp', 'price'])

    # Aggregate the sum of the quantity
    orderbook = orderbook.agg({'quantity': 'sum'})

    # Reset the index
    orderbook = orderbook.reset_index()

    # Create orderbook copies
    orderbook_trimmed = orderbook.copy()
    prices_trimmed = prices.copy()
    
    # Convert the timestamps to datetime objects
    startDate = pd.to_datetime(startDate)
    endDate = pd.to_datetime(endDate)

    # Filter the dataframe to keep only the rows within the desired time range
    orderbook_trimmed = orderbook_trimmed[(orderbook_trimmed['timestamp'] >= startDate) & (orderbook_trimmed['timestamp'] <= endDate)]
    prices_trimmed = prices_trimmed[(prices_trimmed['timestamp'] >= startDate) & (prices_trimmed['timestamp'] <= endDate)]
    
    # Remove the orderbook top percentage of rows from the max
    orderbook_trimmed = orderbook_trimmed[orderbook_trimmed['price'] <= max_price]
    
    # Remove the orderbook bottom percentage of rows from the min 
    orderbook_trimmed = orderbook_trimmed[orderbook_trimmed['price'] >= min_price]
    
    # Create a pivot table for orderbook with the columns reversed
    book_pivot = orderbook_trimmed.pivot(index='timestamp', columns='price', values='quantity')

    book_pivot.index = pd.to_datetime(book_pivot.index)
    book_pivot.index = book_pivot.index.strftime('%H:%M:%S')

    book_pivot = book_pivot.T
    book_pivot = book_pivot.sort_index(ascending=False)
    
    fig, ax = plt.subplots(figsize=(12,6), dpi=1000)
    sns.heatmap(book_pivot, cmap= sns.color_palette('rocket', as_cmap=True), ax=ax, cbar_kws={'label': 'USDC/USDT Limit Order Quantity'})
    ax.set_facecolor("black")
    ax.set_title(title)
    ax.set_xlabel('3/11/2023 | Datetime (UTC)', labelpad=20)
    ax.set_ylabel('')
    ax.set_yticklabels([])
    ax.set_yticks([])
    
    ax2 = ax.twinx()
    ax2.plot(book_pivot.columns, prices_trimmed['mid_price'], color='white', label='Bitfinex USDC/USDT Mid Price')
    ax2.set_ylabel('Price', labelpad=20)
    ax2.yaxis.set_label_position("left")
    ax2.set_ylim(book_pivot.index[-1], book_pivot.index[0])
    ax2.yaxis.tick_left()
    ax2.legend(loc='upper left')
    
    ax2.set_xticks(ax2.get_xticks()[::120])

    plt.show()

### Plot Orderbook

In [ ]:
rounded_order_book_heatmap(ask_book,
                   bid_book,
                   min_price = .85,
                   max_price = 1.05,
                   startDate = '2023-03-11 0:00:00',
                   endDate = '2023-03-11 23:59:00',
                   rounding_factor=.001,
                   title='USDC/USDT Order Book Heatmap - Bitfinex')